In [1]:
from scipy.stats import multivariate_normal

In [2]:
from cpa_utils import *

In [3]:
traces_path = "..\\acquisition\\carto_eB4-Rnd-3-WhiteningAndFullFilter-100000_samples_alternating_same_and_varying\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Alternating-Same-And-Varying-Seeds.mat"
key_path = "..\\acquisition\\carto_eB4-Rnd-3-WhiteningAndFullFilter-100000_samples_alternating_same_and_varying\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Alternating-Same-And-Varying-Seeds.log"

In [4]:
def load_data_alternating_same_varying(traces_path: str, key_path: str, max_traces: int = None) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    traces_dict = sio.loadmat(traces_path, variable_names=[f"data_{i}" for i in range(max_traces)]) if max_traces is not None else sio.loadmat(traces_path)
    traces_size = Counter([traces_dict[k][0, 0][4][:, 0].shape[0] for k in traces_dict.keys() if k.startswith("data_")]).most_common(1)[0][0]
    empty_traces = {k for k in traces_dict.keys() if k.startswith("data_") and traces_dict[k][0, 0][4][:, 0].shape[0] != traces_size}
    traces_tot = np.stack([traces_dict[k][0, 0][4][:, 0] for k in traces_dict.keys() if k.startswith("data_") and k not in empty_traces], axis=0)
    traces_tot = traces_tot.reshape((1, -1, traces_tot.shape[1]))

    inputs_outputs = log_parser.parse(key_path)
    real_keys = np.array([inputs_outputs[1][0][0]])
    real_keys = np.array([[int(c, 16) for c in key] for key in real_keys])
    
    traces_same = traces_tot[:, ::2, :]
    traces_varying = traces_tot[:, 1::2, :]

    seeds_same = np.array([inputs_outputs[0][0][0]])
    seeds_same = np.repeat(seeds_same[:, np.newaxis], traces_same[0].shape[0], axis=1)
    seeds_varying = np.array([inputs_outputs[2 * (int(k[len("data_"):]) - 1)][0][0] for k in traces_dict.keys() if k.startswith("data_") and k not in empty_traces and int(k[len("data_"):]) % 2 == 0])
    seeds_varying = seeds_varying.reshape((1, traces_varying.shape[1]))

    
    assert traces_same.shape[0] == seeds_same.shape[0]
    assert traces_same.shape[1] == seeds_same.shape[1]
    assert traces_varying.shape[0] == seeds_varying.shape[0]
    assert traces_varying.shape[1] == seeds_varying.shape[1]

    return seeds_same, traces_same, seeds_varying, traces_varying, real_keys

seeds_same, traces_same, seeds_varying, traces_varying, real_keys = load_data_alternating_same_varying(traces_path, key_path, max_traces=126728)

In [53]:
arr_rand = np.random.choice(range(traces_varying[0].shape[0]), traces_varying[0].shape[0])
split = arr_rand < 100

traces_train, traces_test = traces_varying[:, ~split], traces_varying[:, split]
seeds_train, seeds_test = seeds_varying[:, ~split], seeds_varying[:, split]

In [54]:
KEY_ALPHABET = list(range(16))

## Profiling stage

### Choosing one point per clock cycle

Arduino Due: 84 MHz
Sampling rate: 2.5 GHz

In [55]:
SAMPLES_PER_CYCLE = int(2500 / 84)
SAMPLES_PER_CYCLE

29

29 points per cycle

#### SOST

In [56]:
def signal_strength_and_template_means_per_index(seeds: np.ndarray, key: List[int], traces: np.ndarray):
    assert seeds.shape[0] == traces.shape[0]
    f = np.zeros((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, int(SAMPLES_PER_CYCLE * np.ceil(traces.shape[1] / SAMPLES_PER_CYCLE))), dtype=np.float32)
    all_m = np.zeros((KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4, BLOCK_WIDTH_B4, len(KEY_ALPHABET), traces.shape[1]), dtype=np.float32)

    for round_idx in range(12, KEYROUND_WIDTH_B4 // BLOCK_WIDTH_B4):
        for block_idx in range(BLOCK_WIDTH_B4):
            card_g = np.zeros((len(KEY_ALPHABET), 1), dtype=np.int32)
            m = all_m[round_idx][block_idx]
            v = np.zeros((len(KEY_ALPHABET), traces.shape[1]), dtype=np.float64)

            for seed, trace in zip(seeds, traces):
                indices, whitening = chacha_random_b4(seed)
                keyround_index = round_idx * BLOCK_WIDTH_B4 + block_idx
                real_k = (key[indices[keyround_index]] + whitening[keyround_index]) % 16
                card_g[real_k][0] += 1
                m[real_k] += trace
                v[real_k] += np.square(trace.astype(np.float64))

            m /= card_g
            v = (v - card_g * m * m) / (card_g - 1)

            for i in range(len(KEY_ALPHABET)):
                for j in range(len(KEY_ALPHABET)):
                    num = m[i] - m[j]
                    den = np.sqrt(v[i] / card_g[i] + v[j] / card_g[j])
                    f[round_idx][block_idx][:traces.shape[1]] += np.square(num / den)
    
    return f, all_m

In [57]:
f, template_means = signal_strength_and_template_means_per_index(seeds_train[0], real_keys[0], traces_train[0])

In [58]:
#interesting_points_per_index = np.array([i + np.argmax(f[..., i:i+SAMPLES_PER_CYCLE], axis=-1) for i in range(0, f.shape[-1], SAMPLES_PER_CYCLE)]).transpose(1, 2, 0)
interesting_points_per_index = np.apply_along_axis(np.argpartition, axis=2, arr=f[:, :, :traces_train.shape[2]], kth=-10)[:, :, -10:]

In [59]:
traces_one_point_per_cycle = traces_train[0][:, interesting_points_per_index].transpose(1, 2, 0, 3)
traces_one_point_per_cycle.shape

(14, 7, 63261, 10)

### Multivariate Gaussian parameters

In [60]:
template_means.shape

(14, 7, 16, 50002)

In [61]:
template_means_reduced = template_means[np.arange(template_means.shape[0])[:, np.newaxis, np.newaxis, np.newaxis], np.arange(template_means.shape[1])[:, np.newaxis, np.newaxis], np.arange(template_means.shape[2])[:, np.newaxis], np.repeat(interesting_points_per_index[:, :, np.newaxis, :], 16, axis=2)]
template_means_reduced.shape

(14, 7, 16, 10)

In [62]:
noise_vector = traces_one_point_per_cycle[:, :, np.newaxis, :, :] - template_means_reduced[:, :, :, np.newaxis, :]
noise_vector.shape

(14, 7, 16, 63261, 10)

In [63]:
covariance_matrices = np.zeros(noise_vector.shape[:3] + (noise_vector.shape[4],) * 2)
for i in range(noise_vector.shape[0]):
    for j in range(noise_vector.shape[1]):
        for k in range(noise_vector.shape[2]):
            covariance_matrices[i, j, k] = np.cov(noise_vector[i, j, k], rowvar=False)
covariance_matrices.shape

(14, 7, 16, 10, 10)

In [64]:
pdfs = np.array([[[multivariate_normal(np.zeros(noise_vector.shape[-1]), covariance_matrices[round_idx, block_idx, key]) for key in range(covariance_matrices.shape[2])] for block_idx in range(covariance_matrices.shape[1])] for round_idx in range(covariance_matrices.shape[0])])
pdfs.shape

(14, 7, 16)

## Extraction stage

In [65]:
traces_test_one_point_per_cycle = traces_test[0][:, interesting_points_per_index].transpose(1, 2, 0, 3)
traces_test_one_point_per_cycle.shape

(14, 7, 101, 10)

In [66]:
noise_test = traces_test_one_point_per_cycle[:, :, np.newaxis, :, :] - template_means_reduced[:, :, :, np.newaxis, :]
noise_test.shape

(14, 7, 16, 101, 10)

In [67]:
probas = np.array([[[[pdfs[round_idx, block_idx, key_guess].pdf(noise_test[round_idx, block_idx, key_guess, trace]) for key_guess in range(pdfs.shape[2])] for block_idx in range(pdfs.shape[1])] for round_idx in range(pdfs.shape[0])] for trace in range(traces_test_one_point_per_cycle.shape[2])])
probas.shape

(101, 14, 7, 16)

In [68]:
np.all(probas == 0)

False

In [98]:
probas_for_each_key_nibble = np.ones((KEY_WIDTH_B4, len(KEY_ALPHABET)))
for i, seed in enumerate(seeds_test[0]):
    indices, whitening = chacha_random_b4(seed)
    for keyround_index in range(KEYROUND_WIDTH_B4):
        key_index = indices.index(keyround_index)
        round_idx = keyround_index // BLOCK_WIDTH_B4
        block_idx = keyround_index % BLOCK_WIDTH_B4
        if round_idx >= 12:
            probas_for_each_key_nibble[key_index] *= probas[i, round_idx, block_idx, (np.arange(len(KEY_ALPHABET)) + whitening[keyround_index]) % 16]

In [99]:
np.all(probas_for_each_key_nibble == 0)

False

In [100]:
print(f"Results: {np.count_nonzero(np.argmax(probas_for_each_key_nibble[np.all(probas_for_each_key_nibble != 1, axis=1)], axis=1) == real_keys[0][np.all(probas_for_each_key_nibble != 1, axis=1)]) / KEY_WIDTH_B4}")

Results: 0.0546875


In [81]:
print(f"At random: {1 / 16}")

At random: 0.0625
